In [6]:
from common import *
from features.selector import Function, Itemgetter, item_check

In [2]:
_TEST_ = False

In [3]:
if __name__=='__main__':
    _TEST_ = True
    interactive(rows=2000, cols=2000)

In [82]:
def fill_template(lines, samp, edit):
    '''Parameters - lines: iterable of lines of the template
                    samp: function float([0,1]) -> float, >0 means choose it, <0 means don't choose it
                    edit: object mapping name -> filter func on ? items
    Yield the processed lines that should join to make a valid query string
    '''
    if hasattr(edit, '_item_'):
        edit_item = edit._item_
    else:
        edit_item = lambda x: x
    
    import re
    whitespace = re.compile(r'\s')
    Form = namedtuple('Form', 're op sep cl')
    forms = [
        Form(re.compile(r'\[.*\S+.*,.*\S+.*\]'), '[', ',', ']'),
        Form(re.compile(r'\(.*\S+.*\|.*\S+.*\)'), '(', '|', ')'),
        Form(re.compile(r'\{.*\S+.*\|.*\S+.*\}'), '{', '|', '}'),
    ]
    
    del_indent = None
    
    for lnum, line in enumerate(lines):
        if line.strip() == '':
            continue
        
        indent = 0
        while whitespace.match(line[indent]):
            assert line[indent] == ' ', f'indents must be space | {lnum}: {line}'
            indent += 1
        if del_indent is not None and indent > del_indent: # still deleting a block
            continue
        del_indent = None
        
        if '=' not in line and '?' not in line:
            assert '@' not in line and '*' not in line
            yield line
            continue
            
        elif '=' in line:
            l0 = line.index('=')
            l1 = line.index('*') if '*' in line else line.index('@') if '@' in line else None
            l2 = line.index('@') if '@' in line else None
            
            x = float(line[l0+1:l1])
            y = samp(x)
            if y <= 0: # delete whole block
                del_indent = indent
                continue
                
            if '*' in line:
                tag = line[l1+1:l2].strip()
                assert hasattr(edit, tag)
                if getattr(edit, tag)() <= 0:
                    del_indent = indent
                    continue
                
            line = line[:l0]+('' if l2 is None else line[l2:])
        
        # at this point the line is definitely in, we just need to process it
        if '@' in line:
            t0 = line.index('@')
            tags = line[t0:].split()
            assert all(tag[0]=='@' for tag in tags)
            tags = [tag[1:] for tag in tags]
            line = line[:t0]
        else:
            tags = []
        
        if '?' in line:
            
            # find which bracket and separator form we have for the ? selections
            match = False
            for f, form in enumerate(forms):
                if form.re.search(line):
                    match = True
                    break
            assert match, f'{lnum}: {line}'
            
            l0, l1 = line.index(forms[f].op), line.index(forms[f].cl)
            items = line[l0+1:l1].split(forms[f].sep)
            
            puts = []
            best_y = -float('inf')
            for item in items:
                assert '?' in item, f'item={repr(item)} | {lnum}: {line}'
                item = item.split('?')
                put, x = item[0], float(item[1])
                x = edit_item(x)
                y = samp(x)
                for tag in (tag for tag in tags if hasattr(edit, tag)):
                    y = getattr(edit, tag)(y, e=put)
                if y > best_y:
                    best_put = put
                if y > 0:
                    puts.append(put)
            if not puts: # if nothing is sampled, add in the one with highest draw
                puts.append(best_put)
            
            yield line[:l0+1] + forms[f].sep.join(puts) + line[l1:]
        else:
            yield line

# Scratcher

In [ ]:
# import json
# #json.dump(list(F.columns), open('feats.json', 'w'))
# feats = json.load(open(top_dir/'work/feats.json', 'r'))

In [47]:
# queries = ['\n'.join(fill_template(fffTemplate.splitlines(), fsamp, None)) for i in range(100)]

In [42]:
# i = 0

In [43]:
# i += 10
# fsamp.r = random.Random(i)
# queries = ['\n'.join(fill_template(fffTemplate.splitlines(), fsamp, None)) for i in range(1)]
# q = queries[0]
# guy = list(filter(FFF.query(q), feats))
# print(len(guy))
# print(q)

In [44]:
# flist = [[print_progress(i), set(filter(FFF.query(q), feats))][1] for i,q in enumerate(queries)]

In [45]:
# for q, l in zip(queries, map(len,flist)):
#     print('='*90)
#     print(l)
#     print(q)